In [ ]:
def modelcomparer(data, radius=10, cutoff=.02, binwidth=2,

                  sigmawidth=4, minI=50, minBeam=.75, EOid=None,

                  plotdata=False, saveall=True, savebefore=True,

                  savebeforeafter=True):



    separated = separator(data)



    modeled = modeler(data, radius, cutoff)

    modeled_RA = copy.deepcopy(modeled['EO_sources_RA'])

    modeled_DEC = copy.deepcopy(modeled['EO_sources_DEC'])

    modeled_I = copy.deepcopy(modeled['EO_sources_I'])

    added_RA = copy.deepcopy(modeled['added_EO_sources_RA'])

    added_DEC = copy.deepcopy(modeled['added_EO_sources_DEC'])

    added_I = copy.deepcopy(modeled['added_EO_sources_I'])

    removed_RA = copy.deepcopy(modeled['removed_EO_sources_RA'])

    removed_DEC = copy.deepcopy(modeled['removed_EO_sources_DEC'])

    removed_I = copy.deepcopy(modeled['removed_EO_sources_I'])



    sought = seeker(data)

    unmodeled_RA = copy.deepcopy(sought['EO_sources_RA'])

    unmodeled_DEC = copy.deepcopy(sought['EO_sources_DEC'])

    unmodeled_I = copy.deepcopy(sought['EO_sources_I'])

    unmodeled_XX = copy.deepcopy(sought['EO_sources_XX'])

    unmodeled_YY = copy.deepcopy(sought['EO_sources_YY'])

    all_RA = copy.deepcopy(sought['all_RA'])

    all_DEC = copy.deepcopy(sought['all_DEC'])

    all_I = copy.deepcopy(sought['all_I'])

    all_XX = copy.deepcopy(sought['all_XX'])

    all_YY = copy.deepcopy(sought['all_YY'])

    all_BEAM = copy.deepcopy(sought['all_BEAM'])



    pixelreplacement = 1e-5

    cmap = matplotlib.cm.get_cmap('gist_heat')

    sigmawidth = sigmawidth * (1. / 3600.)

    binwidth = binwidth * (1. / 3600.)

    kernel = Gaussian2DKernel(stddev=(sigmawidth / binwidth))



    for i in range(len(modeled_RA)):

        for j in range(len(modeled_RA[i])):

            if EOid is not None:

                minI, minBeam = 0, 0

            if (EOid is not None) and (separated['extsources'][i][j]['ID'] != EOid):

                continue

            if (sum(unmodeled_I[i][j]) > minI) and ((sum(unmodeled_XX[i][j]) + sum(unmodeled_YY[i][j])) / sum(unmodeled_I[i][j]) > minBeam):

                print "Currently plotting EOid:", separated['extsources'][i][j]['ID']

                if separated['extsources'][i][j]['RA'] > 180:

                    separated['extsources'][i][j]['RA'] -= 360

                diffuse_RA_total = copy.deepcopy(unmodeled_RA[i][j])

                diffuse_DEC_total = copy.deepcopy(unmodeled_DEC[i][j])

                diffuse_I_total = copy.deepcopy(unmodeled_I[i][j])



                all_framed = framer(all_RA[i], all_DEC[i])

                all_RA_zoom = [all_framed['RA_zoom_min'], all_framed['RA_zoom_max']]

                all_DEC_zoom = [all_framed['DEC_zoom_min'], all_framed['DEC_zoom_max']]

                all_n_bins = min(int(max(all_RA_zoom[1] - all_RA_zoom[0], all_DEC_zoom[1] - all_DEC_zoom[0]) / binwidth), 1024)

                all_RA_total = np.array(all_RA[i])

                all_DEC_total = np.array(all_DEC[i])

                all_I_total = np.array(all_I[i])

                all_XX_total = np.array(all_XX[i])

                all_YY_total = np.array(all_YY[i])

                all_BEAM_total = np.array(all_BEAM[i])

                (all_pixels, all_RA_pixel_centers, all_DEC_pixel_centers) = pixelate(

                    all_RA_zoom, all_DEC_zoom, all_n_bins, all_RA_total, all_DEC_total, all_I_total)

                (all_pixels_contour, all_RA_pixel_centers_contour, all_DEC_pixel_centers_contour) = pixelate(

                    all_RA_zoom, all_DEC_zoom, 50, all_RA_total, all_DEC_total, all_BEAM_total)

                all_pixels[all_pixels == 0] = pixelreplacement

                all_convolved = convolve_fft(all_pixels, kernel)



                unmodeled_framed = framer(unmodeled_RA[i][j], unmodeled_DEC[i][j])

                unmodeled_RA_zoom = [unmodeled_framed['RA_zoom_min'], unmodeled_framed['RA_zoom_max']]

                unmodeled_DEC_zoom = [unmodeled_framed['DEC_zoom_min'], unmodeled_framed['DEC_zoom_max']]

                unmodeled_RA_total = np.array(unmodeled_RA[i][j])

                unmodeled_DEC_total = np.array(unmodeled_DEC[i][j])

                unmodeled_I_total = np.array(unmodeled_I[i][j])

                unmodeled_n_bins = int((unmodeled_framed['RA_zoom_max'] - unmodeled_framed['RA_zoom_min']) / binwidth)

                (unmodeled_pixels, unmodeled_RA_pixel_centers, unmodeled_DEC_pixel_centers) = \

                    pixelate(unmodeled_RA_zoom, unmodeled_DEC_zoom, unmodeled_n_bins,

                             unmodeled_RA_total, unmodeled_DEC_total, unmodeled_I_total)

                unmodeled_pixels[unmodeled_pixels == 0] = pixelreplacement

                unmodeled_pixels = convolve(unmodeled_pixels, kernel)



                modeled_RA_zoom = unmodeled_RA_zoom

                modeled_DEC_zoom = unmodeled_DEC_zoom

                modeled_RA_total = np.array(modeled_RA[i][j])

                modeled_DEC_total = np.array(modeled_DEC[i][j])

                modeled_I_total = np.array(modeled_I[i][j])

                modeled_n_bins = unmodeled_n_bins

                (modeled_pixels, modeled_RA_pixel_centers, modeled_DEC_pixel_centers) = \

                    pixelate(modeled_RA_zoom, modeled_DEC_zoom, modeled_n_bins,

                             modeled_RA_total, modeled_DEC_total, modeled_I_total)

                modeled_pixels[modeled_pixels == 0] = pixelreplacement

                modeled_pixels = convolve(modeled_pixels, kernel)



                added_RA_zoom = unmodeled_RA_zoom

                added_DEC_zoom = unmodeled_DEC_zoom

                added_RA_total = np.array(added_RA[i][j])

                added_DEC_total = np.array(added_DEC[i][j])

                added_I_total = np.array(added_I[i][j])

                added_n_bins = unmodeled_n_bins

                (added_pixels, added_RA_pixel_centers, added_DEC_pixel_centers) = \

                    pixelate(added_RA_zoom, added_DEC_zoom, added_n_bins,

                             added_RA_total, added_DEC_total, added_I_total)

                added_pixels = convolve(added_pixels, kernel)



                removed_RA_zoom = unmodeled_RA_zoom

                removed_DEC_zoom = unmodeled_DEC_zoom

                removed_RA_total = np.array([removed_RA[i][j][k][z]

                                             for k in range(len(removed_RA[i][j]))

                                             for z in range(len(removed_RA[i][j][k]))])

                removed_DEC_total = np.array([removed_DEC[i][j][k][z]

                                              for k in range(len(removed_DEC[i][j]))

                                              for z in range(len(removed_DEC[i][j][k]))])

                removed_I_total = np.array([removed_I[i][j][k][z]

                                            for k in range(len(removed_I[i][j]))

                                            for z in range(len(removed_I[i][j][k]))])

                removed_n_bins = unmodeled_n_bins

                (removed_pixels, removed_RA_pixel_centers, removed_DEC_pixel_centers) = \

                    pixelate(removed_RA_zoom, removed_DEC_zoom, removed_n_bins,

                             removed_RA_total, removed_DEC_total, removed_I_total)

                removed_pixels[removed_pixels == 0] = pixelreplacement

                removed_pixels = convolve(removed_pixels, kernel)



                diffuse_RA_zoom = unmodeled_RA_zoom

                diffuse_DEC_zoom = unmodeled_DEC_zoom

                [diffuse_RA_total.remove(val) for val in removed_RA_total if val in diffuse_RA_total]

                diffuse_RA_total = np.array(diffuse_RA_total)

                [diffuse_DEC_total.remove(val) for val in removed_DEC_total if val in diffuse_DEC_total]

                diffuse_DEC_total = np.array(diffuse_DEC_total)

                [diffuse_I_total.remove(val) for val in removed_I_total if val in diffuse_I_total]

                diffuse_I_total = np.array(diffuse_I_total)

                diffuse_n_bins = unmodeled_n_bins

                (diffuse_pixels, diffuse_RA_pixel_centers, diffuse_DEC_pixel_centers) = \

                    pixelate(diffuse_RA_zoom, diffuse_DEC_zoom, diffuse_n_bins,

                             diffuse_RA_total, diffuse_DEC_total, diffuse_I_total)

                diffuse_pixels[diffuse_pixels == 0] = pixelreplacement

                diffuse_pixels = convolve(diffuse_pixels, kernel)



                ###

                height_ratios = [1, 4, .5, 4]

                width_ratios = [4, .25, 1.5, 4, .25, 1.5, 4, .25]

                wspace = 0.05

                hspace = 0.05

                fig = plt.figure(figsize=(sum(width_ratios) + wspace * (len(width_ratios) - 1),

                                          sum(height_ratios) + hspace * (len(height_ratios) - 1)))

                gs = gridspec.GridSpec(len(height_ratios), len(width_ratios), height_ratios=height_ratios, width_ratios=width_ratios)

                gs.update(left=0, right=1, bottom=0,

                          top=1, wspace=wspace, hspace=hspace)



                bbox_props = dict(boxstyle="square,pad=0.3", fc="white", ec="red", lw=2)



                ###



                fig.suptitle('Clustering of EO Source {} from ObsID {}'.format

                             (separated['extsources'][i][j]['ID'],

                              [int(s) for s in re.findall('\d+', data['filenames'][i])][0],

                              separated['extsources'][i][j]['FREQ']), fontsize=20)



                # Unmodeled Plot

                ax1 = fig.add_subplot(gs[1, 0])

                unmodeled = ax1.imshow(np.transpose(unmodeled_pixels), norm=LogNorm(vmin=pixelreplacement, vmax=1),

                                       origin="lower", interpolation="none", cmap=cmap,

                                       extent=[unmodeled_RA_pixel_centers[0], unmodeled_RA_pixel_centers[-1],

                                               unmodeled_DEC_pixel_centers[0], unmodeled_DEC_pixel_centers[-1]])

                ax1.update({'title': 'Pre-clustered with {} components, {:.4G} Jys'

                            .format(len(unmodeled_RA[i][j]), sum(unmodeled_I[i][j])),

                            'xlabel': 'RA',

                            'ylabel': 'DEC'})

                ax1.grid(True, ls='dashed', alpha=.75)

                ax1.set_xlim(ax1.get_xlim()[::-1])



                # Modeled Plot

                ax2 = fig.add_subplot(gs[1, 3])

                modeled = ax2.imshow(np.transpose(modeled_pixels), norm=LogNorm(vmin=pixelreplacement, vmax=1),

                                     origin="lower", interpolation="none", cmap=cmap,

                                     extent=[modeled_RA_pixel_centers[0], modeled_RA_pixel_centers[-1],

                                             modeled_DEC_pixel_centers[0], modeled_DEC_pixel_centers[-1]])

                ax2.update({'title': 'Clustered with {} components, {:.4G} Jys'

                            .format(len(modeled_RA[i][j]), sum(modeled_I[i][j])),

                            'xlabel': 'RA',

                            'ylabel': 'DEC'})

                if max(removed_RA_total) == 0:

                    no_modeled = ax2.text(np.mean(modeled_RA_pixel_centers),

                                          np.mean(modeled_DEC_pixel_centers),

                                          "No Modeling Occurred", ha="center", va="center",

                                          size=15, bbox=bbox_props)

                    ax2.update({'title': ''})

                ax2.grid(True, ls='dashed', alpha=.75)

                ax2.set_xlim(ax2.get_xlim()[::-1])



                # Modeled Pre-compacted Scatter

                ax3 = fig.add_subplot(gs[1, 6])

                removed = ax3.scatter(removed_RA_total, removed_DEC_total,

                                      c=np.linspace(1, 0, len(removed_I_total)),

                                      cmap=plt.cm.summer, s=2)

                rectangle_length = max((max(removed_RA_total) - min(removed_RA_total)),

                                       (max(removed_DEC_total) - min(removed_DEC_total)))

                ax3.add_patch(patches.Rectangle((min(removed_RA_total) - .1 * rectangle_length,

                                                 min(removed_DEC_total) - .1 * rectangle_length),

                                                1.2 * rectangle_length, 1.2 * rectangle_length,

                                                fill=False, color='coral'))

                ax3.grid(True, ls='dashed', alpha=.75)

                ax3.update({'title': 'Pre-compacted with {} components'.format(len(removed_RA[i][j])),

                            'xlabel': 'RA',

                            'ylabel': 'DEC',

                            'xlim': modeled_RA_zoom,

                            'ylim': modeled_DEC_zoom})

                if max(removed_RA_total) == 0:

                    no_modeled = ax3.text(np.mean(modeled_RA_pixel_centers),

                                          np.mean(modeled_DEC_pixel_centers),

                                          "No Modeling Occurred", ha="center", va="center",

                                          size=15, bbox=bbox_props)

                    ax3.update({'title': ''})

                ax3.set_xlim(ax3.get_xlim()[::-1])

                # Modeled Uncompacted

                ax4 = fig.add_subplot(gs[3, 0])

                all_plot = ax4.imshow(np.transpose(all_convolved), norm=LogNorm(vmin=pixelreplacement),

                                      origin="lower", interpolation="none", cmap=cmap,

                                      extent=[all_RA_pixel_centers[0], all_RA_pixel_centers[-1],

                                              all_DEC_pixel_centers[0], all_DEC_pixel_centers[-1]])

                contour_plot = ax4.contourf(np.transpose(all_pixels_contour), cmap=plt.cm.bone, alpha=.4,

                                            levels=np.linspace(min(all_BEAM_total), max(all_BEAM_total), 6),

                                            extent=[all_RA_pixel_centers[0], all_RA_pixel_centers[-1],

                                                    all_DEC_pixel_centers[0], all_DEC_pixel_centers[-1]])

                location = ax4.scatter(separated['extsources'][i][j]['RA'], separated['extsources'][i][j]['DEC'],

                                       color="cyan", s=60)

                ax4.clabel(contour_plot, fontsize=8,

                           inline=False, colors='white', alpha=1)

                ax4.update({'title': 'Observation',

                            'xlabel': 'RA',

                            'ylabel': 'DEC'})

                ax4.grid(True, ls='dashed', alpha=.75)

                ax4.set_xlim(ax4.get_xlim()[::-1])

                # Modeled Compacted

                ax5 = fig.add_subplot(gs[3, 3])

                diffuse = ax5.imshow(np.transpose(diffuse_pixels), norm=LogNorm(vmin=pixelreplacement, vmax=1),

                                     origin="lower", interpolation="none", cmap=plt.cm.bone,

                                     extent=[unmodeled_RA_pixel_centers[0], unmodeled_RA_pixel_centers[-1],

                                             unmodeled_DEC_pixel_centers[0], unmodeled_DEC_pixel_centers[-1]])

                added = ax5.imshow(np.transpose(added_pixels), norm=LogNorm(vmin=pixelreplacement, vmax=1),

                                   origin="lower", interpolation="none", cmap=cmap,

                                   extent=[modeled_RA_pixel_centers[0], modeled_RA_pixel_centers[-1],

                                           modeled_DEC_pixel_centers[0], modeled_DEC_pixel_centers[-1]])

                ax5.update({'title': 'Compacted with {} componenets, {:.4G} Janskies'.format(len(added_RA[i][j]), sum(added_I[i][j])),

                            'xlabel': 'RA',

                            'ylabel': 'DEC'})

                if max(removed_RA_total) == 0:

                    no_modeled = ax5.text(np.mean(modeled_RA_pixel_centers),

                                          np.mean(modeled_DEC_pixel_centers),

                                          "No Modeling Occurred", ha="center", va="center",

                                          size=15, bbox=bbox_props)

                    ax5.update({'title': ''})

                ax5.grid(True, ls='dashed', alpha=.75)

                ax5.set_xlim(ax5.get_xlim()[::-1])



                # Modeled Pre-compacted Zoom

                ax6 = fig.add_subplot(gs[3, 6])

                removed_zoom = ax6.scatter(removed_RA_total, removed_DEC_total,

                                           c=np.linspace(1, 0, len(removed_I_total)),

                                           cmap=plt.cm.summer, s=20)

                removed_centers = ax6.scatter(added_RA_total, added_DEC_total, c='red', marker='2', s=50)

                ax6.update({'title': 'Pre-compacted Components [Zoomed]',

                            'xlabel': 'RA',

                            'ylabel': 'DEC',

                            'xlim': [min(removed_RA_total) - .1 * rectangle_length,

                                     min(removed_RA_total) + 1.2 * rectangle_length],

                            'ylim': [min(removed_DEC_total) - .1 * rectangle_length,

                                     min(removed_DEC_total) + 1.2 * rectangle_length]})

                if max(removed_RA_total) == 0:

                    no_modeled = ax6.text(np.mean(removed_RA_total),

                                          np.mean(removed_DEC_total),

                                          "No Modeling Occurred", ha="center", va="center",

                                          size=15, bbox=bbox_props)

                    ax6.update({'title': ''})

                ax6.grid(True, ls='dashed', alpha=.75)

                ax6.set_xlim(ax6.get_xlim()[::-1])

                # Colorbars



                unmodeled_cbax = plt.subplot(gs[1, 1])

                unmodeled_cb = Colorbar(ax=unmodeled_cbax, mappable=unmodeled, orientation='vertical', ticklocation='right')

                unmodeled_cb.set_label(r'Janskies', labelpad=10)



                modeled_cbax = plt.subplot(gs[1, 4])

                modeled_cb = Colorbar(ax=modeled_cbax, mappable=modeled, orientation='vertical', ticklocation='right')

                modeled_cb.set_label(r'Janskies', labelpad=10)



                removed_cbax = plt.subplot(gs[1, 7])

                removed_cb = Colorbar(ax=removed_cbax, mappable=removed, orientation='vertical', ticklocation='right')

                removed_cb.set_ticks([1, 0])

                removed_cb.set_ticklabels(['First', 'Last'])



                removed_zoom_cbax = plt.subplot(gs[3, 7])

                removed_zoom_cb = Colorbar(ax=removed_zoom_cbax, mappable=removed_zoom, orientation='vertical', ticklocation='right')

                removed_zoom_cb.set_ticks([1, 0])

                removed_zoom_cb.set_ticklabels(['First', 'Last'])

                if saveall is True:

                    plt.savefig('EO' + '{}radius{}cutoff{}'.format(separated['extsources'][i][j]['ID'], radius, cutoff) + '.png', bbox_inches='tight')

                if savebefore is True:

                    extent = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

                    plt.savefig('EO' + '{}radius{}cutoff{}'.format(separated['extsources'][i][j]['ID'], radius, cutoff) + 'before.png', bbox_inches=extent)

                if savebeforeafter is True:

                    extent = ax5.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

                    plt.savefig('EO' + '{}radius{}cutoff{}'.format(separated['extsources'][i][j]['ID'], radius, cutoff) + 'beforeafter.png', bbox_inches=extent)

                if plotdata is True:

                    plt.show()